# Game Interfaces

This notebook contains two abstract classes: *Game* and *Player* and several example implementations of the Player interface: *HumanPlayer*, *RandomPlayer*, and *MiniMaxPlayer*.

In [1]:
from abc import ABC, abstractmethod

"""
Implementations of this abstract class describe turn-based
games. The field "current_player" stores piece/ID belonging
to the player whose turn is next.
"""
class Game(ABC):
    
    @abstractmethod
    def __init__(self):
        self.current_player = None
    
    @abstractmethod
    def update(self, move):
        pass
    
    @abstractmethod
    def getMoves(self):
        return []
    
    @abstractmethod
    def isGameOver(self):
        return True
    
    @abstractmethod
    def getWinner(self):
        return None
    
    @abstractmethod
    def clone(self):
        pass
    
    @abstractmethod
    def drawGame(self):
        pass
    
    # state evaluation that only differentiates
    # between winning states. A game or player
    # can override this method with something
    # more informative.
    def evaluate(self, player_piece):
        winner = self.getWinner()
        if winner==player_piece:
            return 1
        elif winner!=None:
            return -1
        return 0

In [2]:
from abc import ABC, abstractmethod

class Player(ABC):
    
    def __init__(self, piece):
        self.player_piece = piece
    
    @abstractmethod
    def chooseMove(self, game):
        pass

    # default evaluation method is to use the 
    # game-specific state evaluation that is
    # implemented in the game.
    def evaluate(self, state):
        return state.evaluate(self.player_piece)

# Example Players

Three implementations of the abstract Player class:
- HumanPlayer returns moves chosen by keyboard input
- RandomPlayer returns moves chosen at random.
- MiniMax player chooses moves chosen using the minimax algorithm with alpha-beta pruning.

In [3]:
class HumanPlayer(Player):
    
    def chooseMove(self, game):
        
        # print the available moves
        moveList = game.getMoves()
        for i in range(len(moveList)):
            print(str(i)+": "+str(moveList[i]))
        
        # ask the user to input a move
        move = input("Choose a move!")
        
        # TODO should check for correct input:
        # - is the input an int?
        # - is the input a valid index?
        
        return moveList[int(move)]

In [4]:
import random

class RandomPlayer(Player):
    
    def chooseMove(self, game):
        # return a random move        
        moveList = game.getMoves()
        move = random.randint(0,len(moveList)-1)
        return moveList[int(move)]

In [5]:
class MiniMaxPlayer(Player):
    
    # TODO this must have higher magnitude than any possible state evluation.
    INFINITY = 9999

    def __init__(self, piece, initial_depth):
        super(MiniMaxPlayer, self).__init__(piece)
        self.initial_depth = max(1, initial_depth)
    
    def chooseMove(self, game):
        bestValue, bestMove = self.alphabeta(game, self.initial_depth, -self.INFINITY, self.INFINITY)
        return bestMove
    
    # returns the tuple (value, move) that is the estimated value of the game,
    # and the best move in the game state for its current player.
    def alphabeta(self, game, depth, alpha, beta):
        
        # return evaluation of state
        if depth==0 or game.isGameOver():
            return self.evaluate(game), None
                
        # if its our turn, set best value so far to minus infinity, else positive infinity
        bestValue = -self.INFINITY if (game.current_player == self.player_piece) else self.INFINITY
        bestMove = None
        
        # iterate through moves and choose the one with the best value
        moveList = game.getMoves()       
        for move in moveList:
            
            # evaluate the move
            child = game.clone()
            child.update(move)
            newValue, _ = self.alphabeta(child, depth-1, alpha, beta)

            # check if its value is better
            if game.current_player == self.player_piece:
                # its our turn, so check if the value is higher
                if newValue > bestValue:
                    bestMove = move
                    bestValue = newValue
                alpha = max(alpha, bestValue)
            else:
                # its not our turn, so check if the value is lower
                if newValue < bestValue:
                    bestMove = move
                    bestValue = newValue
                beta = min(beta, bestValue)
                    
            # break out of the loop
            if beta <= alpha:
                break
                
        # return a discounted value to prioritise early rewards
        return bestValue*0.99, bestMove